## Bibliotecas Utilizadas

<div class="text-justify">
 
- **Json:** A biblioteca `json` fornece uma maneira fácil de codificar e decodificar dados no formato JSON (JavaScript Object Notation) em Python. Ela permite converter objetos Python em strings JSON e vice-versa. É amplamente utilizada para trocar dados entre clientes e servidores.

</div>

<div class="text-justify">

- **Shutil:** A biblioteca `shutil` oferece várias operações de alto nível em arquivos e coleções de arquivos. Ela permite copiar, mover, renomear e remover arquivos e diretórios. É útil para gerenciar arquivos de forma programática.

</div>

<div class="text-justify">

- **Random:** A biblioteca `random` gera números pseudo-aleatórios e permite realizar operações como escolher elementos aleatórios de uma lista, embaralhar sequências e gerar amostras aleatórias. Ela é frequentemente usada em simulações e jogos.

</div>

<div class="text-justify">

- **Logging:** A biblioteca `logging` fornece um sistema flexível para registrar eventos de software, mensagens de erro e outros tipos de logs. Ela permite registrar mensagens em diferentes níveis de severidade e enviar esses registros para diversos destinos, como arquivos e consoles.

  - **logging.config:** A biblioteca `logging.config` estende a funcionalidade de `logging` permitindo a configuração de registros por meio de arquivos de configuração. Ela facilita a definição de configurações complexas de registro sem a necessidade de código adicional.

</div>

<div class="text-justify">

- **Sys:** A biblioteca `sys` fornece acesso a algumas variáveis e funções interativas do interpretador Python. Ela permite manipular o ambiente de execução, como argumentos de linha de comando e fluxo de entrada/saída padrão. É útil para scripts que interagem diretamente com o sistema.

</div>

<div class="text-justify">

- **Pathlib:** A biblioteca `pathlib` fornece classes para manipulação de caminhos do sistema de arquivos de forma orientada a objetos. Ela facilita operações de leitura, escrita e navegação em diretórios de maneira intuitiva. É uma alternativa moderna à biblioteca `os.path`.

</div>



In [26]:
import json
import shutil
import random
import argparse
import logging
import logging.config
import sys
from pathlib import Path

# Separação do Dataset

As funções a seguir separam o dataset que temos em Treinamento, Validação e Teste. Para isso, temos as seguintes funções:

- **split_dataset**: Divide um conjunto de dados COCO em conjuntos de treinamento, validação e teste com base em proporções definidas. Carrega as anotações a partir de um arquivo JSON, valida as proporções fornecidas, embaralha aleatoriamente as imagens e as distribui conforme as proporções especificadas. Além disso, cria subconjuntos COCO para cada um dos conjuntos de dados (treinamento, validação, teste) e salva os arquivos JSON correspondentes em diretórios específicos, mantendo a estrutura de anotações e categorias.

- **copy_images**: Copia as imagens selecionadas para um diretório especificado. Para cada imagem, a função constrói o caminho completo de origem e destino, realiza a cópia da imagem e registra logs de sucesso ou erro durante o processo, garantindo que todas as imagens necessárias sejam corretamente transferidas para os diretórios de treinamento, validação e teste.

In [27]:
# Configuração básica do logging para capturar logs de informação
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def copy_images(images, src_dir, dest_dir):
    """
    Copia as imagens selecionadas para um diretório especificado.
    
    Args:
        images (list): Lista de dicionários contendo informações das imagens.
        src_dir (str): Diretório de origem das imagens.
        dest_dir (str): Diretório de destino das imagens.
    """
    for image in images:
        try:
            # Construção do caminho completo da imagem de origem
            src_path = Path(src_dir) / image['file_name']
            # Construção do caminho completo da imagem de destino
            dest_path = Path(dest_dir) / src_path.name
            # Copia a imagem do diretório de origem para o diretório de destino
            shutil.copy(src_path, dest_path)
            # Log de sucesso na cópia da imagem
            logger.info(f"Copiado com sucesso de {src_path} para {dest_path}")
        except Exception as e:
            # Log de erro se a cópia falhar
            logger.error(f"Falha ao copiar {src_path} para {dest_path}: {e}")

def split_dataset(images_dir, labels_json_path, output_dir, train_ratio=0.75, val_ratio=0.1):
    """
    Divide um conjunto de dados COCO em conjuntos de treinamento, validação e teste com base em determinadas proporções.
    
    Args:
        images_dir (str): Diretório contendo as imagens.
        labels_json_path (str): Caminho do arquivo JSON contendo as anotações COCO.
        output_dir (str): Diretório onde os dados divididos serão armazenados.
        train_ratio (float): Proporção do conjunto de treinamento.
        val_ratio (float): Proporção do conjunto de validação.
    """
    try:
        # Log informando o início do carregamento das anotações
        logger.info("Carregando as anotações feitas em COCO...")
        # Carregamento das anotações COCO a partir do arquivo JSON
        with open(labels_json_path, 'r') as f:
            coco_data = json.load(f)
    except FileNotFoundError:
        # Log de erro se o arquivo JSON não for encontrado
        logger.error(f"Arquivo não encontrado: {labels_json_path}")
        return
    except json.JSONDecodeError:
        # Log de erro se o arquivo JSON estiver inválido
        logger.error(f"Arquivo JSON Invalido: {labels_json_path}")
        return

    # Convertendo caminhos de diretórios para objetos Path
    images_dir = Path(images_dir)
    output_dir = Path(output_dir)

    if not images_dir.exists():
        # Log de erro se o diretório das imagens não existir
        logger.error(f"Diretorio das Imagens não existe: {images_dir}")
        return

    # Extração dos detalhes das imagens e anotações do conjunto de dados COCO
    images = coco_data.get('images', [])
    annotations = coco_data.get('annotations', [])

    # Validação das proporções de treinamento e validação
    if not (0 < train_ratio < 1 and 0 <= val_ratio < 1 and train_ratio + val_ratio <= 1):
        # Log de erro se as proporções forem inválidas
        logger.error("Proporções de treinamento/validação invalidas.")
        return

    # Embaralhamento aleatório das imagens
    random.shuffle(images)
    total_images = len(images)
    # Cálculo do índice final do conjunto de treinamento
    train_end = int(total_images * train_ratio)
    # Cálculo do índice final do conjunto de validação
    val_end = train_end + int(total_images * val_ratio)

    # Divisão das imagens em conjuntos de treinamento, validação e teste
    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]

    def filter_annotations(images_set):
        """
        Filtra as anotações para corresponder ao conjunto de imagens fornecido.
        
        Args:
            images_set (list): Lista de imagens para filtrar as anotações.
            
        Returns:
            list: Lista de anotações filtradas.
        """
        image_ids = {image['id'] for image in images_set}
        return [annotation for annotation in annotations if annotation['image_id'] in image_ids]

    def create_coco_subset(images, annotations):
        """
        Cria um subconjunto COCO com as imagens e anotações fornecidas.
        
        Args:
            images (list): Lista de imagens.
            annotations (list): Lista de anotações.
            
        Returns:
            dict: Subconjunto COCO contendo imagens, anotações e categorias.
        """
        return {
            'images': images,
            'annotations': annotations,
            'categories': coco_data['categories']
        }

    # Iteração sobre os tipos de conjuntos de dados (treinamento, validação, teste)
    for type, images_set in zip(["train", "val", "test"], [train_images, val_images, test_images]):
        try:
            # Criação do caminho de saída para as imagens
            images_output_path = output_dir / "images" / type
            images_output_path.mkdir(parents=True, exist_ok=True)

            # Criação do caminho de saída para as anotações
            labels_output_path = output_dir / "labels" / type
            labels_output_path.mkdir(parents=True, exist_ok=True)

            # Copia das imagens para o diretório de saída correspondente
            copy_images(images_set, images_dir, images_output_path)

            # Criação do arquivo JSON COCO para o conjunto de dados atual
            coco_file = create_coco_subset(images_set, filter_annotations(images_set))
            with open(labels_output_path / "coco.json", 'w') as file:
                json.dump(coco_file, file, indent=4)
            # Log de sucesso na criação do conjunto de dados
            logger.info(f"Conjunto de dados para {type} salvo com sucesso.")
        except Exception as e:
            # Log de erro se houver falha no processamento do conjunto de dados
            logger.error(f"Falha ao processar dados para {type}: {e}")


# Executando a Separação do Dataset

<div style="text-align: justify;">
 
- **origin**: Define o caminho para a pasta mãe, onde o diretório principal do workshop está localizado.

- **images_dir**: Especifica o caminho para a pasta que contém as imagens do dataset.

- **coco_json_path**: Aponta para o arquivo JSON que contém as anotações COCO do dataset.

- **output_dir**: Indica o caminho para a pasta onde os dados separados serão armazenados.

- **train_ratio**: Define a proporção de dados que serão utilizados para treinamento (neste caso, 75%).

- **val_ratio**: Define a proporção de dados que serão utilizados para validação (neste caso, 10%).

</div>

<div style="text-align: justify;"> <br>

Ao chamar a função `split_dataset` com os caminhos e proporções definidos, o dataset será dividido em três conjuntos: <br>

- **Treinamento**: 75% dos dados

- **Validação**: 10% dos dados

- **Teste**: O restante dos dados (15%)

A função não só realiza a divisão das imagens, mas também copia as imagens para os diretórios apropriados e gera arquivos JSON COCO correspondentes para cada subconjunto. Isso garante que os dados estejam prontos para serem usados em treinamentos e avaliações de modelos de detecção de objetos.

</div>


In [ ]:
# Caminho para a pasta mãe, object_detection_workshop
origin = Path.cwd().parent

# Caminho para a pasta com as imagens do dataset
images_dir = origin / "Dados/Dataset/images"

# Caminho para a pasta do .json com as anotações do dataset
coco_json_path = origin / "Dados/Dataset/annotations/instances_default.json"

# Caminho para a pasta com os dados de saída
output_dir = origin / "Dados/Saida"

# Proporção do conjunto de treinamento
train_ratio = 0.75

# Proporção do conjunto de validação
val_ratio = 0.1

# Chamando a função split_dataset, ela separa o dataset em treino, teste e validação
split_dataset(images_dir, coco_json_path, output_dir, train_ratio, val_ratio)
